# We are going to write a script that will web scrape flight information for multiple flights from the official Heathrow Airport website:

---

### Import the necessary libraries

In [1]:
# this is the library that we will use to create break times in order to mimic human behaviour
import time
from getpass import getpass


# Juicy stuff- these are the Classes we will use for interaction with a webpage:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


# library for interacting with the operating system
import os

# you know pandas it's your best buddy
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

# library for directory location:

import pathlib
from os.path import join

#Ignore warning -- Some methods are going to be deprecated and I didn't change all (mainly in the function scrapper)
import warnings
warnings.filterwarnings('ignore')

from playsound import playsound
from datetime import datetime
import mysql.connector

---

### Filepaths

In [86]:
date = input("What date: ")
file_path_arr = f"/Users/martynas/Desktop/Ironhack/Project - Final/Href Links Collection/Heathrow Href Links/arrival_links_september_{date}.txt"
file_path_dep = f"/Users/martynas/Desktop/Ironhack/Project - Final/Href Links Collection/Heathrow Href Links/departure_links_september_{date}.txt"
file_path_arr_data = f"/Users/martynas/Desktop/Ironhack/Project - Final/Data Collection/Heathrow Data/arrivals_september_{date}.xlsx"
file_path_dep_data = f"/Users/martynas/Desktop/Ironhack/Project - Final/Data Collection/Heathrow Data/departures_september_{date}.xlsx"

What date: 12


### Load a driver and the website

In [92]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [93]:
driver.get('https://www.heathrow.com/departures')

### Click cookies

In [94]:
try:
    cookie_button = driver.find_element(By.ID, "tealium_ensCloseBanner")
    cookie_button.click()
    time.sleep(2.5)
except:
    print("Cookies already accepted.")

### Departures

In [135]:
html = driver.page_source
soup = BeautifulSoup(html)
base_url = "https://www.heathrow.com"
div = soup.find('div', class_='airline-listing-table')
links = div.find_all('a', class_='airline-listing-line-item')
full_urls = [base_url + link['href'] for link in links]
date = input("What date: ")
file_path_dep = f"/Users/martynas/Desktop/Ironhack/Project - Final/Href Links Collection/Heathrow Href Links/departure_links_september_{date}.txt"

# Write the URLs to the specified file
with open(file_path_dep, 'w') as file:
    for url in full_urls:
        file.write(url + '\n')

What date: 15


### Append the new links

In [174]:
html = driver.page_source
soup = BeautifulSoup(html)
div = soup.find('div', class_='airline-listing-table')
links = div.find_all('a', class_='airline-listing-line-item')
full_urls = [base_url + link['href'] for link in links]
with open(file_path_dep, 'r') as file:
    existing_urls = set(file.read().splitlines())
unique_new_urls = [url for url in full_urls if url not in existing_urls]
with open(file_path_dep, 'a') as file:
    for url in unique_new_urls:
        file.write(url + '\n')
if unique_new_urls:
    print("URLs have been appended")
else:
    print("No new unique URLs to append.")
print(len(existing_urls))

No new unique URLs to append.
659


### Arrivals

In [214]:
html = driver.page_source
soup = BeautifulSoup(html)
base_url = "https://www.heathrow.com"
div = soup.find('div', class_='airline-listing-table')
links = div.find_all('a', class_='airline-listing-line-item')
full_urls = [base_url + link['href'] for link in links]
date = input("What date: ")
file_path_arr = f"/Users/martynas/Desktop/Ironhack/Project - Final/Href Links Collection/Heathrow Href Links/arrival_links_september_{date}.txt"

# Write the URLs to the specified file
with open(file_path_arr, 'w') as file:
    for url in full_urls:
        file.write(url + '\n')

What date: 15


### Append the new links

In [252]:
html = driver.page_source
soup = BeautifulSoup(html)
div = soup.find('div', class_='airline-listing-table')
links = div.find_all('a', class_='airline-listing-line-item')
full_urls = [base_url + link['href'] for link in links]
with open(file_path_arr, 'r') as file:
    existing_urls = set(file.read().splitlines())
unique_new_urls = [url for url in full_urls if url not in existing_urls]
with open(file_path_arr, 'a') as file:
    for url in unique_new_urls:
        file.write(url + '\n')
if unique_new_urls:
    print("URLs have been appended")
else:
    print("No new unique URLs to append.")
print(len(existing_urls))

No new unique URLs to append.
673


### Open the links file into a list

In [88]:
with open(file_path_dep, 'r') as file:
    href_list_dep = [line.strip() for line in file]

In [89]:
with open(file_path_arr, 'r') as file:
    href_list_arr = [line.strip() for line in file]

In [5]:
len(href_list_arr)

692

In [6]:
len(href_list_dep)

691

### Test for departure elements

In [90]:
driver.get(href_list_dep[25])

In [230]:
flight_nr = driver.find_element(By.CSS_SELECTOR,"h4[class^='heading-4-dark m0 flight-number']").text
flight_nr

'BA1432'

In [231]:
route_codes = driver.find_element(By.CSS_SELECTOR,"div[class^='stops flex flex-justify cursor-pointer']").text
route_codes

'LHR\nEDI'

In [232]:
airline = driver.find_element(By.CSS_SELECTOR,"a[class^='cta tertiary']").text
airline

'British Airways'

In [233]:
status = driver.find_element(By.CSS_SELECTOR,"div[class^='status lg-mb2 green']").text
status

'DEPARTED'

In [234]:
arrival_details = driver.find_element(By.CSS_SELECTOR,"div[class^='show-more-container sm-bdr-t sm-pt4']").text
arrival_details

'Arrive Edinburgh (EDI)\nDate\nMon, 02 Sep\nTime\n08:05\n08:25\nTerminal\n-\nThis flight also operates under the following flight numbers under codeshare arrangements:QF3551, QR6264\nContact British Airways\nView Flight map\nView all Flights'

In [235]:
details = driver.find_element(By.CSS_SELECTOR,"div[class^='flight-detail relative show-flight-details']").text
details

'Depart London (LHR)\nDate\nMon, 02 Sep\nTime\n07:15\n07:00\nTerminal\n5\nGate\n-\nCheck-in zone\nC\nCheck-in open time\n05:00'

### Test for arrival elements

In [222]:
driver.get(href_list_arr[2])

In [223]:
flight_nr = driver.find_element(By.CSS_SELECTOR,"h4[class^='heading-4-dark m0 flight-number']").text
flight_nr

'BA074'

In [224]:
route_codes = driver.find_element(By.CSS_SELECTOR,"div[class^='stops flex flex-justify ']").text
route_codes

'LOS\nLHR'

In [225]:
airline = driver.find_element(By.CSS_SELECTOR,"a[class^='cta tertiary']").text
airline

'British Airways'

In [228]:
status = driver.find_element(By.CSS_SELECTOR,"div[class^='status lg-mb2 green']").text
status

'LANDED'

In [227]:
arrival_details = driver.find_element(By.CSS_SELECTOR,"div[class^='show-more-container sm-bdr-t sm-pt4']").text
arrival_details

'Arrive London (LHR)\nDate\nMon, 02 Sep\nTime\n05:02\n05:25\nTerminal\n5\nBaggage\nBags delivered on belt 5\nThis flight also operates under the following flight numbers under codeshare arrangements:AY5904\nContact British Airways\nView all Flights'

In [226]:
details = driver.find_element(By.CSS_SELECTOR,"div[class^='flight-detail relative show-flight-details']").text
details

'Depart Lagos (LOS)\nDate\nSun, 01 Sep\nTime\n22:50\nTerminal\nI'

## Run to create a database

### Departures

### Appending existing df

In [253]:
flight_data_dep = []

date_of_scrape = input("What Day: ")
for_loading = 0

file_path_dep_data = f"/Users/martynas/Desktop/Ironhack/Project - Final/Data Collection/Heathrow Data/departures_september_{date_of_scrape}.xlsx"
file_path_dep = f"/Users/martynas/Desktop/Ironhack/Project - Final/Href Links Collection/Heathrow Href Links/departure_links_september_{date_of_scrape}.txt"

error_path = '/Users/martynas/Desktop/Ironhack/Project - Final/error.mp3'
sound_path = '/Users/martynas/Desktop/Ironhack/Project - Final/ding.mp3'

# Load existing data
try:
    existing_dataframe = pd.read_excel(file_path_dep_data)
    print(f"Loaded existing dataframe with {len(existing_dataframe)} rows.")
except FileNotFoundError:
    existing_dataframe = pd.DataFrame(columns=["Flight Number", "Route Codes", "Airline", "Status", "Details 1", "Details 2"])
    print("No existing file found. Creating a new dataframe.")

# Read the links
with open(file_path_dep, 'r') as file:
    href_list_dep = [line.strip() for line in file]

# Scrape data for the next 90 links
try:
    for link in href_list_dep:

        driver.get(link)
        time.sleep(2)

        try:
            cookie_button = driver.find_element(By.ID, "tealium_ensCloseBanner")
            cookie_button.click()
            time.sleep(2.5)
        except:
            pass

        driver.execute_script("document.body.style.zoom='100%'")
        driver.execute_script("document.body.style.zoom='67%'")
        time.sleep(1)

        # Extracting information
        try:
            flight_nr = driver.find_element(By.CSS_SELECTOR,"h4[class^='heading-4-dark m0 flight-number']").text
        except:
            flight_nr = None
        try:
            try:
                route_codes = driver.find_element(By.CSS_SELECTOR,"div[class^='stops flex flex-justify cursor-pointer']").text
            except:
                route_codes = driver.find_element(By.CSS_SELECTOR,"div[class^='stops flex flex-justify ']").text
        except:
            route_codes = None
        try:
            airline = driver.find_element(By.CSS_SELECTOR,"a[class^='cta tertiary']").text
        except:
            airline = None
        try:
            try:
                status = driver.find_element(By.CSS_SELECTOR,"div[class^='status lg-mb2 green']").text
            except:
                status = driver.find_element(By.CSS_SELECTOR,"div[class^='status lg-mb2 red']").text
        except:
            status = None
        try:
            arrival_details = driver.find_element(By.CSS_SELECTOR,"div[class^='show-more-container sm-bdr-t sm-pt4']").text
        except:
            arrival_details = None
        try:
            details = driver.find_element(By.CSS_SELECTOR,"div[class^='flight-detail relative show-flight-details']").text
        except:
            details = None

        flight_idx_list_dep = [flight_nr, route_codes, airline, status, details, arrival_details]
        flight_data_dep.append(flight_idx_list_dep)

        for_loading += 1
        print(f"{for_loading + 0} out of {len(href_list_dep)} scraped")

    # Convert new data to DataFrame
    new_dataframe = pd.DataFrame(flight_data_dep,
                                 columns=["Flight Number", "Route Codes", "Airline", "Status", "Details 1", "Details 2"])

    # Append new data to existing data
    updated_dataframe = pd.concat([existing_dataframe, new_dataframe], ignore_index=True)

    # Save updated data back to the Excel file
    updated_dataframe.to_excel(file_path_dep_data, sheet_name='Flights', index=False)
    print(f"Updated dataframe saved with {len(updated_dataframe)} rows.")

    playsound(sound_path)
except:
    playsound(error_path)
    print("no working")

What Day: 12
No existing file found. Creating a new dataframe.
1 out of 689 scraped
2 out of 689 scraped
3 out of 689 scraped
4 out of 689 scraped
5 out of 689 scraped
6 out of 689 scraped
7 out of 689 scraped
8 out of 689 scraped
9 out of 689 scraped
10 out of 689 scraped
11 out of 689 scraped
12 out of 689 scraped
13 out of 689 scraped
14 out of 689 scraped
15 out of 689 scraped
16 out of 689 scraped
17 out of 689 scraped
18 out of 689 scraped
19 out of 689 scraped
20 out of 689 scraped
21 out of 689 scraped
22 out of 689 scraped
23 out of 689 scraped
24 out of 689 scraped
25 out of 689 scraped
26 out of 689 scraped
27 out of 689 scraped
28 out of 689 scraped
29 out of 689 scraped
30 out of 689 scraped
31 out of 689 scraped
32 out of 689 scraped
33 out of 689 scraped
34 out of 689 scraped
35 out of 689 scraped
36 out of 689 scraped
37 out of 689 scraped
38 out of 689 scraped
39 out of 689 scraped
40 out of 689 scraped
41 out of 689 scraped
42 out of 689 scraped
43 out of 689 scraped


360 out of 689 scraped
361 out of 689 scraped
362 out of 689 scraped
363 out of 689 scraped
364 out of 689 scraped
365 out of 689 scraped
366 out of 689 scraped
367 out of 689 scraped
368 out of 689 scraped
369 out of 689 scraped
370 out of 689 scraped
371 out of 689 scraped
372 out of 689 scraped
373 out of 689 scraped
374 out of 689 scraped
375 out of 689 scraped
376 out of 689 scraped
377 out of 689 scraped
378 out of 689 scraped
379 out of 689 scraped
380 out of 689 scraped
381 out of 689 scraped
382 out of 689 scraped
383 out of 689 scraped
384 out of 689 scraped
385 out of 689 scraped
386 out of 689 scraped
387 out of 689 scraped
388 out of 689 scraped
389 out of 689 scraped
390 out of 689 scraped
391 out of 689 scraped
392 out of 689 scraped
393 out of 689 scraped
394 out of 689 scraped
395 out of 689 scraped
396 out of 689 scraped
397 out of 689 scraped
398 out of 689 scraped
399 out of 689 scraped
400 out of 689 scraped
401 out of 689 scraped
402 out of 689 scraped
403 out of 

### Arrivals

### Appending existing df

In [274]:
flight_data_arr = []
date_of_scrape = input("What Date: ")
for_loading = 0

file_path_arr_data = f"/Users/martynas/Desktop/Ironhack/Project - Final/Data Collection/Heathrow Data/arrivals_september_{date_of_scrape}.xlsx"
file_path_arr = f"/Users/martynas/Desktop/Ironhack/Project - Final/Href Links Collection/Heathrow Href Links/arrival_links_september_{date_of_scrape}.txt"

sound_path = '/Users/martynas/Desktop/Ironhack/Project - Final/ding.mp3'
error_path = '/Users/martynas/Desktop/Ironhack/Project - Final/error.mp3'

# Load existing data
try:
    existing_arrival_dataframe = pd.read_excel(file_path_arr_data)
    print(f"Loaded existing dataframe with {len(existing_arrival_dataframe)} rows.")
except FileNotFoundError:
    existing_arrival_dataframe = pd.DataFrame(columns=["Flight Number", "Route Codes", "Airline", "Status", "Details 1", "Details 2"])
    print("No existing file found. Creating a new dataframe.")

# Read the links
with open(file_path_arr, 'r') as file:
    href_list_arr = [line.strip() for line in file]

# Scrape data for the next 30 links
try:
    for link in href_list_arr:

        driver.get(link)
        time.sleep(2)

        try:
            cookie_button = driver.find_element(By.ID, "tealium_ensCloseBanner")
            cookie_button.click()
            time.sleep(2.5)
        except:
            pass

        driver.execute_script("document.body.style.zoom='100%'")
        driver.execute_script("document.body.style.zoom='67%'")
        time.sleep(1)

        # Extracting information
        try:
            flight_nr = driver.find_element(By.CSS_SELECTOR,"h4[class^='heading-4-dark m0 flight-number']").text
        except:
            flight_nr = None
        try:
            try:
                route_codes = driver.find_element(By.CSS_SELECTOR,"div[class^='stops flex flex-justify cursor-pointer']").text
            except:
                route_codes = driver.find_element(By.CSS_SELECTOR,"div[class^='stops flex flex-justify ']").text
        except:
            route_codes = None
        try:
            airline = driver.find_element(By.CSS_SELECTOR,"a[class^='cta tertiary']").text
        except:
            airline = None
        try:
            try:
                status = driver.find_element(By.CSS_SELECTOR,"div[class^='status lg-mb2 green']").text
            except:
                status = driver.find_element(By.CSS_SELECTOR,"div[class^='status lg-mb2 red']").text
        except:
            status = None
        try:
            arrival_details = driver.find_element(By.CSS_SELECTOR,"div[class^='show-more-container sm-bdr-t sm-pt4']").text
        except:
            arrival_details = None
        try:
            details = driver.find_element(By.CSS_SELECTOR,"div[class^='flight-detail relative show-flight-details']").text
        except:
            details = None

        flight_idx_list_arr = [flight_nr, route_codes, airline, status, details, arrival_details]

        flight_data_arr.append(flight_idx_list_arr)

        for_loading += 1
        print(f"{for_loading + 0} out of {len(href_list_arr)} scraped")

    # Convert new data to DataFrame
    new_arrival_dataframe = pd.DataFrame(flight_data_arr,
                                         columns=["Flight Number", "Route Codes", "Airline", "Status", "Details 1", "Details 2"])

    # Append new data to existing data
    updated_arrival_dataframe = pd.concat([existing_arrival_dataframe, new_arrival_dataframe], ignore_index=True)

    # Save updated data back to the Excel file
    updated_arrival_dataframe.to_excel(file_path_arr_data, sheet_name='Flights', index=False)
    print(f"Updated dataframe saved with {len(updated_arrival_dataframe)} rows.")

    playsound(sound_path)
except:
    playsound(error_path)
    print("Error")

What Date: 12
No existing file found. Creating a new dataframe.
1 out of 688 scraped
2 out of 688 scraped
3 out of 688 scraped
4 out of 688 scraped
5 out of 688 scraped
6 out of 688 scraped
7 out of 688 scraped
8 out of 688 scraped
9 out of 688 scraped
10 out of 688 scraped
11 out of 688 scraped
12 out of 688 scraped
13 out of 688 scraped
14 out of 688 scraped
15 out of 688 scraped
16 out of 688 scraped
17 out of 688 scraped
18 out of 688 scraped
19 out of 688 scraped
20 out of 688 scraped
21 out of 688 scraped
22 out of 688 scraped
23 out of 688 scraped
24 out of 688 scraped
25 out of 688 scraped
26 out of 688 scraped
27 out of 688 scraped
28 out of 688 scraped
29 out of 688 scraped
30 out of 688 scraped
31 out of 688 scraped
32 out of 688 scraped
33 out of 688 scraped
34 out of 688 scraped
35 out of 688 scraped
36 out of 688 scraped
37 out of 688 scraped
38 out of 688 scraped
39 out of 688 scraped
40 out of 688 scraped
41 out of 688 scraped
42 out of 688 scraped
43 out of 688 scraped

360 out of 688 scraped
361 out of 688 scraped
362 out of 688 scraped
363 out of 688 scraped
364 out of 688 scraped
365 out of 688 scraped
366 out of 688 scraped
367 out of 688 scraped
368 out of 688 scraped
369 out of 688 scraped
370 out of 688 scraped
371 out of 688 scraped
372 out of 688 scraped
373 out of 688 scraped
374 out of 688 scraped
375 out of 688 scraped
376 out of 688 scraped
377 out of 688 scraped
378 out of 688 scraped
379 out of 688 scraped
380 out of 688 scraped
381 out of 688 scraped
382 out of 688 scraped
383 out of 688 scraped
384 out of 688 scraped
385 out of 688 scraped
386 out of 688 scraped
387 out of 688 scraped
388 out of 688 scraped
389 out of 688 scraped
390 out of 688 scraped
391 out of 688 scraped
392 out of 688 scraped
393 out of 688 scraped
394 out of 688 scraped
395 out of 688 scraped
396 out of 688 scraped
397 out of 688 scraped
398 out of 688 scraped
399 out of 688 scraped
400 out of 688 scraped
401 out of 688 scraped
402 out of 688 scraped
403 out of 

### Close Driver

In [91]:
driver.close()

---

# Organising Unstructured Data

### Open and Concat departures

In [67]:
folder_path = '/Users/martynas/Desktop/Ironhack/Project - Final/Data Collection/Heathrow Data'

# Initialize an empty list to store the dataframes
dfs = []

# Loop through the file names and load each Excel file into a dataframe
for day in range(1, 32):
    try:
        file_name = f'departures_september_{day:02}.xlsx'  # Formatted to have leading zero
        file_path = os.path.join(folder_path, file_name)  # Get full file path

        df = pd.read_excel(file_path)
        dfs.append(df)
    except:
        pass

# Concatenate all dataframes into one
data = pd.concat(dfs, ignore_index=True)

# Display the final dataframe (or use it for further analysis)
display(data)
display(data.shape)

,Flight Number,Route Codes,Airline,Status,Details 1,Details 2
0,OS458,LHR\nVIE,Austrian,DEPARTED,"Depart London (LHR)\nDate\nMon, 02 Sep\nTime\n...","Arrive Vienna (VIE)\nDate\nMon, 02 Sep\nTime\n..."
1,LX345,LHR\nZRH,Swiss International Airlines,DEPARTED,"Depart London (LHR)\nDate\nMon, 02 Sep\nTime\n...","Arrive Zurich (ZRH)\nDate\nMon, 02 Sep\nTime\n..."
2,BA420,LHR\nAGP,British Airways,DEPARTED,"Depart London (LHR)\nDate\nMon, 02 Sep\nTime\n...","Arrive Malaga (AGP)\nDate\nMon, 02 Sep\nTime\n..."
3,AF1381,LHR\nCDG,Air France,DEPARTED,"Depart London (LHR)\nDate\nMon, 02 Sep\nTime\n...","Arrive Paris (CDG)\nDate\nMon, 02 Sep\nTime\n0..."
4,BA1430,LHR\nEDI,British Airways,DEPARTED,"Depart London (LHR)\nDate\nMon, 02 Sep\nTime\n...","Arrive Edinburgh (EDI)\nDate\nMon, 02 Sep\nTim..."
...,...,...,...,...,...,...
5438,EI725,LHR\nORK,Aer Lingus,DEPARTED,"Depart London (LHR)\nDate\nMon, 09 Sep\nTime\n...","Arrive Cork (ORK)\nDate\nMon, 09 Sep\nTime\n23..."
5439,BA073,LHR\nAUH,British Airways,DEPARTED,"Depart London (LHR)\nDate\nMon, 09 Sep\nTime\n...","Arrive Abu Dhabi (AUH)\nDate\nTue, 10 Sep\nTim..."
5440,VS411,LHR\nLOS,Virgin Atlantic,DEPARTED,"Depart London (LHR)\nDate\nMon, 09 Sep\nTime\n...","Arrive Lagos (LOS)\nDate\nTue, 10 Sep\nTime\n0..."
5441,AM008,LHR\nMEX,Aeromexico,DEPARTED,"Depart London (LHR)\nDate\nMon, 09 Sep\nTime\n...","Arrive Mexico City (MEX)\nDate\nTue, 10 Sep\nT..."


(5443, 6)

---

### Departures
#### Open New Day

In [254]:
date_new = input("What date: ")
data = pd.read_excel(f"/Users/martynas/Desktop/Ironhack/Project - Final/Data Collection/Heathrow Data/departures_september_{date_new}.xlsx")

What date: 12


#### Route Codes

In [255]:
route_splits = data['Route Codes'].str.split('\n')
data['Origin'] = route_splits.str[0]
data['Intersect'] = route_splits.apply(lambda x: x[1] if isinstance(x, list) and len(x) == 3 else np.nan)
data['Destination'] = route_splits.str[-1]
data = data.drop(columns=['Route Codes'])

#### Details 1

In [256]:
def fix_shifted_columns_dep_details_1(row):
    if row['Departure Time Scheduled'] == 'Terminal':
        row['Check In Time'] = row['Check Text 2']  # Move Check Text 2 to Check In Time
        row['Check Text 2'] = row['Check In']       # Move Check In to Check Text 2
        row['Check In'] = row['Check Text']         # Move Check Text to Check In
        row['Check Text'] = row['Gate']             # Move Gate to Check Text
        row['Gate'] = row['Gate Text']              # Move Gate Text to Gate
        row['Gate Text'] = row['Terminal']          # Move Terminal to Gate Text
        row['Terminal'] = row['Terminal Text']      # Move Terminal Text to Terminal
        row['Terminal Text'] = row['Departure Time Scheduled']# Move Time Scheduled to Terminal Text
        row['Departure Time Scheduled'] = row['Time Departed']# Move Time Departed to Time Scheduled
    return row

In [257]:
columns_for_details = ['Depart', 'Date Text', 'Date', 'Time Text',
                       'Time Departed', 'Departure Time Scheduled', 'Terminal Text',
                       'Terminal', 'Gate Text', 'Gate', 'Check Text',
                       'Check In', 'Check Text 2', 'Check In Time']
split_details = data['Details 1'].str.split('\n', expand=True)
split_details.columns = columns_for_details
split_details = split_details.apply(fix_shifted_columns_dep_details_1, axis=1)
data = pd.concat([data, split_details], axis=1)
data = data.drop(columns=['Details 1'])
columns_to_drop = ['Date Text', 'Time Text', 'Terminal Text', 'Gate Text', 'Check Text', 'Check Text 2']
data = data.drop(columns=columns_to_drop)

#### Details 2

In [258]:
def fix_shifted_columns_dep_details_2(row):
    if row['Arrive Time Scheduled'] == 'Terminal':  # Check if 'Time Scheduled' contains 'Terminal'
        row['Arrival Terminal'] = row['Terminal Text']      # Move Gate Text to Terminal
        row['Terminal Text'] = row['Arrive Time Scheduled']  # Move Terminal to Terminal Text
        row['Arrive Time Scheduled'] = row['Time Arrived']  # Move Time Departed to Time Scheduled
    return row

In [259]:
columns_for_details_2 = ['Arrive', 'Date Text', 'Date Arrived', 'Time Text',
                       'Time Arrived', 'Arrive Time Scheduled', 'Terminal Text',
                       'Arrival Terminal', 'Info_1', 'Info_2', 'Info_3', 'Info_4']
split_details_2 = data['Details 2'].str.split('\n', expand=True)
split_details_2.columns = columns_for_details_2
split_details_2 = split_details_2.apply(fix_shifted_columns_dep_details_2, axis=1)
columns_to_drop_2 = ['Date Text', 'Time Text', 'Terminal Text', 'Info_1', 'Info_2', 'Info_3', 'Info_4']
split_details_2 = split_details_2.drop(columns=columns_to_drop_2)
data = pd.concat([data, split_details_2], axis=1)
data = data.drop(columns=['Details 2'])

#### Date

In [260]:
def day_to_number(day_name):
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    return days.index(day_name) + 1

def convert_date(date_str, year=2024):
    try:
        # Parse the input string and assume the year
        date_obj = datetime.strptime(f"{date_str}, {year}", '%a, %d %b, %Y')
        # Format the date to the desired format
        return date_obj.strftime('%d/%m/%Y')
    except ValueError:
        # Return NaN if the date string doesn't match the expected format
        return np.nan

# Apply the functions, handling cases where 'Date' might be NaN or not a string
data['Day Code'] = data['Date'].apply(lambda x: day_to_number(x.split(',')[0]) if isinstance(x, str) else np.nan)
data['Date Real'] = data['Date'].apply(lambda x: convert_date(x, year=2024) if isinstance(x, str) else np.nan)
data['Date Arrive'] = data['Date Arrived'].apply(lambda x: convert_date(x, year=2024) if isinstance(x, str) and x != '-' else np.nan)

# Drop the original 'Date' and 'Date Arrived' columns, and rename 'Date Real'
data = data.drop(columns=['Date', 'Date Arrived'])
data.rename(columns={'Date Real': 'Date Depart'}, inplace=True)

#### Cities

In [261]:
data['Departure City'] = data['Depart'].str.extract(r'Depart (\w+)')
data['Destination City'] = data['Arrive'].str.extract(r'Arrive (.+?) \(')
drop_cols = ['Depart', 'Arrive']
data = data.drop(columns=drop_cols)

#### Countries and Timezones

In [262]:
city_country_df = pd.read_excel('/Users/martynas/Desktop/Ironhack/Project - Final/city_country_mapping.xlsx')
time_zone_df = pd.read_excel('/Users/martynas/Desktop/Ironhack/Project - Final/time_zone_offsets.xlsx')
data = pd.merge(data, city_country_df, left_on='Destination City', right_on='Destination City', how='left')
data = pd.merge(data, time_zone_df, left_on='Country', right_on='Country', how='left')

#### Drop Duplicates

In [263]:
display(data.shape)
data.drop_duplicates(inplace=True)
data.shape

(689, 22)

(679, 22)

#### Flight Durations

In [264]:
# Create a copy of time_data with necessary columns
time_data = data[['Time Departed', 'Time Arrived', 'Time Zone Offset']].copy()

# Convert time columns to datetime with a fixed date
fixed_date = '2000-01-01'  # arbitrary fixed date
time_data['Time Departed'] = pd.to_datetime(fixed_date + ' ' + time_data['Time Departed'], format='%Y-%m-%d %H:%M', errors='coerce')
time_data['Time Arrived'] = pd.to_datetime(fixed_date + ' ' + time_data['Time Arrived'], format='%Y-%m-%d %H:%M', errors='coerce')

# Apply the time zone offset only to the departure times
time_data['Time Departed'] += pd.to_timedelta(time_data['Time Zone Offset'], unit='h')

# Calculate flight duration: handle overnight flights
time_data['Flight Duration Seconds'] = (time_data['Time Arrived'] - time_data['Time Departed']).dt.total_seconds()

# Adjust for cases where arrival time is before departure time (next day arrival)
time_data.loc[time_data['Flight Duration Seconds'] < 0, 'Flight Duration Seconds'] += 86400  # Add 24 hours in seconds

# Function to convert seconds to HH:MM:SS format
def format_duration(seconds):
    if pd.isna(seconds) or seconds < 0:
        return '00:00:00'
    hours, remainder = divmod(int(seconds), 3600)
    minutes, seconds = divmod(remainder, 60)
    return f'{hours:02}:{minutes:02}:{seconds:02}'

# Apply the function to convert durations to HH:MM:SS format
time_data['Flight Duration'] = time_data['Flight Duration Seconds'].apply(format_duration)

# Merge this duration back into the original data
data['Flight Duration'] = time_data['Flight Duration']

# Drop temporary columns if needed
time_data.drop(columns=['Flight Duration Seconds'], inplace=True)

#### Airline Estimated Flight Durations

In [265]:
# Create a copy of time_data with necessary columns
time_data = data[['Departure Time Scheduled', 'Arrive Time Scheduled', 'Time Zone Offset']].copy()

# Convert time columns to datetime with a fixed date
fixed_date = '2000-01-01'  # arbitrary fixed date
time_data['Departure Time Scheduled'] = pd.to_datetime(fixed_date + ' ' + time_data['Departure Time Scheduled'], format='%Y-%m-%d %H:%M', errors='coerce')
time_data['Arrive Time Scheduled'] = pd.to_datetime(fixed_date + ' ' + time_data['Arrive Time Scheduled'], format='%Y-%m-%d %H:%M', errors='coerce')

# Apply the time zone offset only to the departure times
time_data['Departure Time Scheduled'] += pd.to_timedelta(time_data['Time Zone Offset'], unit='h')

# Calculate flight duration: handle overnight flights
time_data['Estimated Flight Duration Seconds'] = (time_data['Arrive Time Scheduled'] - time_data['Departure Time Scheduled']).dt.total_seconds()

# Adjust for cases where arrival time is before departure time (next day arrival)
time_data.loc[time_data['Estimated Flight Duration Seconds'] < 0, 'Estimated Flight Duration Seconds'] += 86400  # Add 24 hours in seconds

# Function to convert seconds to HH:MM:SS format
def format_duration(seconds):
    if pd.isna(seconds) or seconds < 0:
        return '00:00:00'
    hours, remainder = divmod(int(seconds), 3600)
    minutes, seconds = divmod(remainder, 60)
    return f'{hours:02}:{minutes:02}:{seconds:02}'

# Apply the function to convert durations to HH:MM:SS format
time_data['Estimated Flight Duration'] = time_data['Estimated Flight Duration Seconds'].apply(format_duration)

# Merge this duration back into the original data
data['Estimated Flight Duration'] = time_data['Estimated Flight Duration']

# Drop temporary columns if needed
time_data.drop(columns=['Estimated Flight Duration Seconds'], inplace=True)

#### Departure Time Difference

In [266]:
# Existing code for calculating time differences
data['Date Depart'] = data['Date Depart'].astype(str)
data['Scheduled DateTime'] = pd.to_datetime(data['Date Depart'] + ' ' + data['Departure Time Scheduled'])
data['Actual Departure DateTime'] = pd.to_datetime(data['Date Depart'] + ' ' + data['Time Departed'])
data['Time Difference'] = data['Actual Departure DateTime'] - data['Scheduled DateTime']
data['Departure Time Difference'] = (data['Time Difference'].dt.total_seconds() / 60)
data['Departure Time Difference'] = data['Departure Time Difference'].fillna(0).round().astype(int)

# Mask for large positive or negative values
mask_positive = data['Departure Time Difference'] > 900
mask_negative = data['Departure Time Difference'] < -900

# Adjust positive differences (flight departed next day)
data.loc[mask_positive, 'Departure Time Difference'] = data['Departure Time Difference'] - 1440

# Adjust negative differences (flight was scheduled on previous day but delayed to next)
data.loc[mask_negative, 'Departure Time Difference'] = data['Departure Time Difference'] + 1440

# Clean up unnecessary columns
data.drop(columns=['Time Difference', 'Actual Departure DateTime', 'Scheduled DateTime'], inplace=True)

#### Arrival Time Difference

In [267]:
# Define a function to handle datetime conversion and errors
def safe_to_datetime(date_str, time_str):
    try:
        # Create the full datetime string
        datetime_str = f"{date_str} {time_str}"
        # Convert to datetime, catching errors
        return pd.to_datetime(datetime_str, format='%d/%m/%Y %H:%M', errors='coerce')
    except Exception as e:
        print(f"Error processing {date_str} {time_str}: {e}")
        return pd.NaT

# Clean up and convert time columns
data['Date Arrive'] = data['Date Arrive'].astype(str)
data['Arrive Time Scheduled'] = data['Arrive Time Scheduled'].astype(str)
data['Time Arrived'] = data['Time Arrived'].astype(str)

# Apply safe conversion
data['Scheduled DateTime'] = data.apply(lambda row: safe_to_datetime(row['Date Arrive'], row['Arrive Time Scheduled']), axis=1)
data['Actual Arrival DateTime'] = data.apply(lambda row: safe_to_datetime(row['Date Arrive'], row['Time Arrived']), axis=1)

# Calculate time differences
data['Time Difference'] = data['Actual Arrival DateTime'] - data['Scheduled DateTime']
data['Arrival Time Difference'] = (data['Time Difference'].dt.total_seconds() / 60)
data['Arrival Time Difference'] = data['Arrival Time Difference'].fillna(0).round().astype(int)

# Mask for large positive or negative values
mask_positive = data['Arrival Time Difference'] > 900
mask_negative = data['Arrival Time Difference'] < -900

# Adjust positive differences (flight departed next day)
data.loc[mask_positive, 'Arrival Time Difference'] = data['Arrival Time Difference'] - 1440

# Adjust negative differences (flight was scheduled on previous day but delayed to next)
data.loc[mask_negative, 'Arrival Time Difference'] = data['Arrival Time Difference'] + 1440

# Clean up unnecessary columns
data.drop(columns=['Time Difference', 'Actual Arrival DateTime', 'Scheduled DateTime'], inplace=True)

#### Estimation Time Difference

In [268]:
def hms_to_seconds(hms):
    try:
        h, m, s = map(int, hms.split(':'))
        return h * 3600 + m * 60 + s
    except Exception as e:
        return 0

# Convert 'Flight Duration' and 'Estimated Flight Duration' to total seconds
data['Flight Duration Seconds'] = data['Flight Duration'].apply(hms_to_seconds)
data['Estimated Flight Duration Seconds'] = data['Estimated Flight Duration'].apply(hms_to_seconds)

# Calculate the difference in seconds
data['Estimation Difference Seconds'] = data['Estimated Flight Duration Seconds'] - data['Flight Duration Seconds']

# Convert the difference to minutes
data['Estimation Difference Minutes'] = data['Estimation Difference Seconds'] / 60

# Round to the nearest whole number if needed
data['Estimation Difference Minutes'] = data['Estimation Difference Minutes'].round().astype(int)

# Drop temporary columns if needed
data.drop(columns=['Flight Duration Seconds', 'Estimated Flight Duration Seconds', 'Estimation Difference Seconds'], inplace=True)

#### Departure Status

In [269]:
def classify_departure_time_difference(diff):
    if diff == 0:
        return 'On Time'
    elif diff > 0:
        return 'Delayed'
    else:
        return 'Early'

# Apply the function to create the new column
data['Departure Status'] = data['Departure Time Difference'].apply(classify_departure_time_difference)
data.loc[data['Status'] == 'CANCELLED', 'Departure Status'] = 'Cancelled'

#### Arrival Status

In [270]:
def classify_arrival_time_difference(diff):
    if diff == 0:
        return 'On Time'
    elif diff > 0:
        return 'Delayed'
    else:
        return 'Early'

# Apply the function to create the new column
data['Arrival Status'] = data['Arrival Time Difference'].apply(classify_arrival_time_difference)
data.loc[data['Status'] == 'CANCELLED', 'Arrival Status'] = 'Cancelled'

## Export to SQL

In [271]:
# Step 1: Connect to the MySQL Database
connection = mysql.connector.connect(
    host="127.0.0.1",         # Database host (use "localhost" if running locally)
    user="root",              # Your MySQL username
    password="Lolablades1.",   # Your MySQL password
    database="Flights"         # Database name
)

cursor = connection.cursor()

# Step 2: Double-check the connection
if connection.is_connected():
    print("Connection to the database is successful!")
    
    # Step 3: Run a simple query to verify the connection
    try:
        cursor.execute("SHOW TABLES;")
        tables = cursor.fetchall()
        print("Tables in the database:", tables)
    except mysql.connector.Error as err:
        print(f"Error: {err}")
else:
    print("Connection failed.")

Connection to the database is successful!
Tables in the database: [('airlines_list',), ('berlin_arrivals',), ('berlin_departures',), ('departure_ai',), ('heathrow_arrivals',), ('heathrow_departures',), ('lisbon_arrivals',), ('lisbon_departures',)]


In [272]:
display(data.shape)
data = data[~((data['Flight Duration'] == '00:00:00') & 
              (data['Origin'].isna()) & 
              (data['Country'].isna()))]
data = data[data['Airline'] != 'Find my terminal']
data['Date Depart'] = pd.to_datetime(data['Date Depart'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
data['Date Arrive'] = pd.to_datetime(data['Date Arrive'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
data['Time Departed'] = data['Time Departed'].replace(np.nan, None)
data['Time Arrived'] = data['Time Arrived'].replace(np.nan, None)
data.shape

(679, 29)

(678, 29)

In [273]:
# Step 3: Create the table (if it doesn't exist) with additional columns adjusted for SQL types
create_table_query = """
CREATE TABLE IF NOT EXISTS heathrow_departures (
    id INT AUTO_INCREMENT PRIMARY KEY,
    flight_number VARCHAR(10),                  -- Flight number as VARCHAR
    airline VARCHAR(100),                       -- Airline as VARCHAR
    status VARCHAR(20),                         -- Flight status as VARCHAR
    origin VARCHAR(4),                          -- Origin airport IATA code as VARCHAR
    intersect VARCHAR(4),                       -- Intersect airport IATA code as VARCHAR
    destination VARCHAR(4),                     -- Destination airport IATA code as VARCHAR
    time_departed TIME,                         -- Time of departure as TIME
    departure_time_scheduled TIME,              -- Scheduled departure time as TIME
    terminal VARCHAR(8),                        -- Terminal as VARCHAR
    gate VARCHAR(8),                            -- Gate as VARCHAR
    check_in VARCHAR(8),                        -- Check-in area as VARCHAR
    check_in_time TIME,                         -- Check-in time as TIME
    time_arrived TIME,                          -- Time of arrival as TIME
    arrive_time_scheduled TIME,                 -- Scheduled arrival time as TIME
    arrival_terminal VARCHAR(8),                -- Arrival terminal as VARCHAR
    day_code INT,                               -- Day code as INT
    date_depart DATE,                           -- Departure date as DATE
    date_arrive DATE,                           -- Arrival date as DATE
    departure_city VARCHAR(100),                -- Departure city as VARCHAR
    destination_city VARCHAR(100),              -- Destination city as VARCHAR
    country VARCHAR(100),                       -- Country as VARCHAR
    time_zone_offset DECIMAL(5,1),              -- Time zone offset as DECIMAL
    flight_duration TIME,                       -- Flight duration as TIME
    estimated_flight_duration TIME,             -- Estimated flight duration as TIME
    departure_time_difference INT,              -- Difference in departure time as INT
    arrival_time_difference INT,                -- Difference in arrival time as INT
    estimation_difference_minutes INT,          -- Estimation difference in minutes as INT
    departure_status VARCHAR(20),               -- Departure status as VARCHAR
    arrival_status VARCHAR(20)                  -- Arrival status as VARCHAR
);
"""
cursor.execute(create_table_query)

# Step 4: Prepare SQL query to insert data
insert_query = """
INSERT INTO heathrow_departures (flight_number, airline, status, origin, intersect, destination, time_departed,
                     departure_time_scheduled, terminal, gate, check_in, check_in_time, time_arrived, 
                     arrive_time_scheduled, arrival_terminal, day_code, date_depart, date_arrive, 
                     departure_city, destination_city, country, time_zone_offset, flight_duration, 
                     estimated_flight_duration, departure_time_difference, arrival_time_difference, 
                     estimation_difference_minutes, departure_status, arrival_status)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Step 5: Iterate over the DataFrame and insert each row into the table
for i, row in data.iterrows():
    try:
        cursor.execute(insert_query, (
            row['Flight Number'], 
            row['Airline'],
            row['Status'], 
            row['Origin'], 
            row['Intersect'], 
            row['Destination'], 
            row['Time Departed'], 
            row['Departure Time Scheduled'], 
            row['Terminal'], 
            row['Gate'], 
            row['Check In'], 
            row['Check In Time'], 
            row['Time Arrived'], 
            row['Arrive Time Scheduled'], 
            row['Arrival Terminal'], 
            row['Day Code'], 
            row['Date Depart'], 
            row['Date Arrive'], 
            row['Departure City'], 
            row['Destination City'], 
            row['Country'], 
            row['Time Zone Offset'], 
            row['Flight Duration'], 
            row['Estimated Flight Duration'], 
            row['Departure Time Difference'], 
            row['Arrival Time Difference'], 
            row['Estimation Difference Minutes'], 
            row['Departure Status'], 
            row['Arrival Status']
        ))
    except mysql.connector.Error as err:
        print(f"Error at row {i}: {err}")

# Step 6: Commit the transaction and close the connection
connection.commit()

cursor.close()
connection.close()

print("CSV file uploaded successfully!")

CSV file uploaded successfully!


---

### Open and Concat arrivals

In [108]:
folder_path = '/Users/martynas/Desktop/Ironhack/Project - Final/Data Collection/Heathrow Data'

# Initialize an empty list to store the dataframes
dfs = []

# Loop through the file names and load each Excel file into a dataframe
for day in range(1, 32):
    try:
        file_name = f'arrivals_september_{day:02}.xlsx'
        file_path = os.path.join(folder_path, file_name)

        df = pd.read_excel(file_path)
        dfs.append(df)
    except:
        pass

# Concatenate all dataframes into one
data = pd.concat(dfs, ignore_index=True)

# Display the final dataframe (or use it for further analysis)
display(data)
display(data.shape)

,Flight Number,Route Codes,Airline,Status,Details 1,Details 2
0,VS450,JNB\nLHR,Virgin Atlantic,LANDED,"Depart Johannesburg (JNB)\nDate\nSun, 01 Sep\n...","Arrive London (LHR)\nDate\nMon, 02 Sep\nTime\n..."
1,QF009,PER\nLHR,Qantas,LANDED,"Depart Perth (PER)\nDate\nSun, 01 Sep\nTime\n1...","Arrive London (LHR)\nDate\nMon, 02 Sep\nTime\n..."
2,BA074,LOS\nLHR,British Airways,LANDED,"Depart Lagos (LOS)\nDate\nSun, 01 Sep\nTime\n2...","Arrive London (LHR)\nDate\nMon, 02 Sep\nTime\n..."
3,BA016,SYD\nSIN\nLHR,British Airways,LANDED,"Depart Sydney (SYD)\nDate\nSun, 01 Sep\nTime\n...","Arrive London (LHR)\nDate\nMon, 02 Sep\nTime\n..."
4,BA056,JNB\nLHR,British Airways,LANDED,"Depart Johannesburg (JNB)\nDate\nSun, 01 Sep\n...","Arrive London (LHR)\nDate\nMon, 02 Sep\nTime\n..."
...,...,...,...,...,...,...
5480,LH2484,MUC\nLHR,Lufthansa,LANDED,"Depart Munich (MUC)\nDate\nMon, 09 Sep\nTime\n...","Arrive London (LHR)\nDate\nMon, 09 Sep\nTime\n..."
5481,BA357,NCE\nLHR,British Airways,LANDED,"Depart Nice (NCE)\nDate\nMon, 09 Sep\nTime\n21...","Arrive London (LHR)\nDate\nMon, 09 Sep\nTime\n..."
5482,EI388,SNN\nLHR,Aer Lingus,LANDED,"Depart Shannon (SNN)\nDate\nMon, 09 Sep\nTime\...","Arrive London (LHR)\nDate\nMon, 09 Sep\nTime\n..."
5483,BA371,MRS\nLHR,British Airways,LANDED,"Depart Marseille (MRS)\nDate\nMon, 09 Sep\nTim...","Arrive London (LHR)\nDate\nMon, 09 Sep\nTime\n..."


(5485, 6)

#### Open New Day

In [30]:
date_new = input("What date: ")
data = pd.read_excel(f"/Users/martynas/Desktop/Ironhack/Project - Final/Data Collection/Heathrow Data/arrivals_september_{date_new}.xlsx")

What date: 10


#### Route Codes

In [31]:
route_splits = data['Route Codes'].str.split('\n')
data['Origin'] = route_splits.str[0]
data['Intersect'] = route_splits.apply(lambda x: x[1] if isinstance(x, list) and len(x) == 3 else np.nan)
data['Destination'] = route_splits.str[-1]
data = data.drop(columns=['Route Codes'])

#### Details 1

In [32]:
def fix_shifted_columns_arr_details_1(row):
    if row['Departure Time Scheduled'] == 'Terminal':
        row['Terminal'] = row['Terminal Text']      # Move Terminal Text to Terminal
        row['Terminal Text'] = row['Departure Time Scheduled']# Move Time Scheduled to Terminal Text
        row['Departure Time Scheduled'] = row['Time Departed']# Move Time Departed to Time Scheduled
    return row

In [33]:
columns_for_details = ['Depart', 'Date Text', 'Date', 'Time Text',
                       'Time Departed', 'Departure Time Scheduled', 'Terminal Text',
                       'Terminal']
split_details = data['Details 1'].str.split('\n', expand=True)
split_details.columns = columns_for_details
split_details = split_details.apply(fix_shifted_columns_arr_details_1, axis=1)
data = pd.concat([data, split_details], axis=1)
data = data.drop(columns=['Details 1'])
columns_to_drop = ['Date Text', 'Time Text', 'Terminal Text']
data = data.drop(columns=columns_to_drop)

#### Details 2

In [34]:
def fix_shifted_columns_arr_details_2(row):
    if row['Arrive Time Scheduled'] == 'Terminal':  # Check if 'Time Scheduled' contains 'Terminal'
        row['Arrival Terminal'] = row['Terminal Text']      # Move Gate Text to Terminal
        row['Terminal Text'] = row['Arrive Time Scheduled']  # Move Terminal to Terminal Text
        row['Arrive Time Scheduled'] = row['Time Arrived']  # Move Time Departed to Time Scheduled
    return row

In [35]:
columns_for_details_2 = ['Arrive', 'Date Text', 'Date Arrive', 'Time Text',
                       'Time Arrived', 'Arrive Time Scheduled', 'Terminal Text',
                       'Arrival Terminal', 'Info_1', 'Info_2', 'Info_3', 'Info_4', 'Info_5']
split_details_2 = data['Details 2'].str.split('\n', expand=True)
split_details_2.columns = columns_for_details_2
split_details_2 = split_details_2.apply(fix_shifted_columns_arr_details_2, axis=1)
columns_to_drop_2 = ['Date Text', 'Time Text', 'Terminal Text', 'Info_1', 'Info_2', 'Info_3', 'Info_4', 'Info_5']
split_details_2 = split_details_2.drop(columns=columns_to_drop_2)
data = pd.concat([data, split_details_2], axis=1)
data = data.drop(columns=['Details 2'])

#### Date

In [36]:
def day_to_number(day_name):
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    return days.index(day_name) + 1

def convert_date(date_str, year=2024):
    try:
        # Parse the input string and assume the year
        date_obj = datetime.strptime(f"{date_str}, {year}", '%a, %d %b, %Y')
        # Format the date to the desired format
        return date_obj.strftime('%d/%m/%Y')
    except ValueError:
        # Return NaN if the date string doesn't match the expected format
        return np.nan

# Apply the functions, handling cases where 'Date' might be NaN or not a string
data['Day Code'] = data['Date Arrive'].apply(lambda x: day_to_number(x.split(',')[0]) if isinstance(x, str) else np.nan)
data['Date Real'] = data['Date Arrive'].apply(lambda x: convert_date(x, year=2024) if isinstance(x, str) else np.nan)
data['Date Depart'] = data['Date'].apply(lambda x: convert_date(x, year=2024) if isinstance(x, str) and x != '-' else np.nan)

# Drop the original 'Date' and 'Date Arrived' columns, and rename 'Date Real'
data = data.drop(columns=['Date', 'Date Arrive'])
data.rename(columns={'Date Real': 'Date Arrive'}, inplace=True)

#### Cities

In [37]:
data['Departure City'] = data['Depart'].str.extract(r'Depart (.+?) \(')
data['Destination City'] = data['Arrive'].str.extract(r'Arrive (.+?) \(')
drop_cols = ['Depart', 'Arrive']
data = data.drop(columns=drop_cols)

#### Countries and Timezones

In [38]:
city_country_df = pd.read_excel('/Users/martynas/Desktop/Ironhack/Project - Final/city_country_mapping.xlsx')
city_country_df.rename(columns={'Destination City': 'Departure City'}, inplace=True)
data = pd.merge(data, city_country_df, left_on='Departure City', right_on='Departure City', how='left')

#### Drop Duplicates

In [39]:
display(data.shape)
data.drop_duplicates(inplace=True)
data.shape

(694, 18)

(656, 18)

#### Flight Durations

In [40]:
# Create a copy of time_data with necessary columns
time_data = data[['Time Departed', 'Time Arrived']].copy()

# Convert time columns to datetime with a fixed date
fixed_date = '2000-01-01'  # arbitrary fixed date
time_data['Time Departed'] = pd.to_datetime(fixed_date + ' ' + time_data['Time Departed'], format='%Y-%m-%d %H:%M', errors='coerce')
time_data['Time Arrived'] = pd.to_datetime(fixed_date + ' ' + time_data['Time Arrived'], format='%Y-%m-%d %H:%M', errors='coerce')

# Calculate flight duration: handle overnight flights
time_data['Flight Duration Seconds'] = (time_data['Time Arrived'] - time_data['Time Departed']).dt.total_seconds()

# Adjust for cases where arrival time is before departure time (next day arrival)
time_data.loc[time_data['Flight Duration Seconds'] < 0, 'Flight Duration Seconds'] += 86400  # Add 24 hours in seconds

# Function to convert seconds to HH:MM:SS format
def format_duration(seconds):
    if pd.isna(seconds) or seconds < 0:
        return '00:00:00'
    hours, remainder = divmod(int(seconds), 3600)
    minutes, seconds = divmod(remainder, 60)
    return f'{hours:02}:{minutes:02}:{seconds:02}'

# Apply the function to convert durations to HH:MM:SS format
time_data['Flight Duration'] = time_data['Flight Duration Seconds'].apply(format_duration)

# Merge this duration back into the original data
data['Flight Duration'] = time_data['Flight Duration']

# Drop temporary columns if needed
time_data.drop(columns=['Flight Duration Seconds'], inplace=True)

#### Airline Estimated Flight Durations

In [41]:
# Create a copy of time_data with necessary columns
time_data = data[['Departure Time Scheduled', 'Arrive Time Scheduled']].copy()

# Convert time columns to datetime with a fixed date
fixed_date = '2000-01-01'  # arbitrary fixed date
time_data['Departure Time Scheduled'] = pd.to_datetime(fixed_date + ' ' + time_data['Departure Time Scheduled'], format='%Y-%m-%d %H:%M', errors='coerce')
time_data['Arrive Time Scheduled'] = pd.to_datetime(fixed_date + ' ' + time_data['Arrive Time Scheduled'], format='%Y-%m-%d %H:%M', errors='coerce')

# Calculate flight duration: handle overnight flights
time_data['Estimated Flight Duration Seconds'] = (time_data['Arrive Time Scheduled'] - time_data['Departure Time Scheduled']).dt.total_seconds()

# Adjust for cases where arrival time is before departure time (next day arrival)
time_data.loc[time_data['Estimated Flight Duration Seconds'] < 0, 'Estimated Flight Duration Seconds'] += 86400  # Add 24 hours in seconds

# Function to convert seconds to HH:MM:SS format
def format_duration(seconds):
    if pd.isna(seconds) or seconds < 0:
        return '00:00:00'
    hours, remainder = divmod(int(seconds), 3600)
    minutes, seconds = divmod(remainder, 60)
    return f'{hours:02}:{minutes:02}:{seconds:02}'

# Apply the function to convert durations to HH:MM:SS format
time_data['Estimated Flight Duration'] = time_data['Estimated Flight Duration Seconds'].apply(format_duration)

# Merge this duration back into the original data
data['Estimated Flight Duration'] = time_data['Estimated Flight Duration']

# Drop temporary columns if needed
time_data.drop(columns=['Estimated Flight Duration Seconds'], inplace=True)

#### Departure Time Difference

In [42]:
# Define a function to handle datetime conversion and errors
def safe_to_datetime(date_str, time_str):
    try:
        # Create the full datetime string
        datetime_str = f"{date_str} {time_str}"
        # Convert to datetime, catching errors
        return pd.to_datetime(datetime_str, format='%d/%m/%Y %H:%M', errors='coerce')
    except Exception as e:
        print(f"Error processing {date_str} {time_str}: {e}")
        return pd.NaT

# Clean up and convert time columns
data['Date Depart'] = data['Date Depart'].astype(str)
data['Departure Time Scheduled'] = data['Departure Time Scheduled'].astype(str)
data['Time Departed'] = data['Time Departed'].astype(str)

# Apply safe conversion
data['Scheduled DateTime'] = data.apply(lambda row: safe_to_datetime(row['Date Depart'], row['Departure Time Scheduled']), axis=1)
data['Actual Departure DateTime'] = data.apply(lambda row: safe_to_datetime(row['Date Depart'], row['Time Departed']), axis=1)

# Calculate time differences
data['Time Difference'] = data['Actual Departure DateTime'] - data['Scheduled DateTime']
data['Departure Time Difference'] = (data['Time Difference'].dt.total_seconds() / 60)
data['Departure Time Difference'] = data['Departure Time Difference'].fillna(0).round().astype(int)

# Mask for large positive or negative values
mask_positive = data['Departure Time Difference'] > 900
mask_negative = data['Departure Time Difference'] < -900

# Adjust positive differences (flight departed next day)
data.loc[mask_positive, 'Departure Time Difference'] = data['Departure Time Difference'] - 1440

# Adjust negative differences (flight was scheduled on previous day but delayed to next)
data.loc[mask_negative, 'Departure Time Difference'] = data['Departure Time Difference'] + 1440

# Clean up unnecessary columns
data.drop(columns=['Time Difference', 'Actual Departure DateTime', 'Scheduled DateTime'], inplace=True)


#### Arrival Time Difference

In [43]:
# Define a function to handle datetime conversion and errors
def safe_to_datetime(date_str, time_str):
    try:
        # Create the full datetime string
        datetime_str = f"{date_str} {time_str}"
        # Convert to datetime, catching errors
        return pd.to_datetime(datetime_str, format='%d/%m/%Y %H:%M', errors='coerce')
    except Exception as e:
        print(f"Error processing {date_str} {time_str}: {e}")
        return pd.NaT

# Clean up and convert time columns
data['Date Arrive'] = data['Date Arrive'].astype(str)
data['Arrive Time Scheduled'] = data['Arrive Time Scheduled'].astype(str)
data['Time Arrived'] = data['Time Arrived'].astype(str)

# Apply safe conversion
data['Scheduled DateTime'] = data.apply(lambda row: safe_to_datetime(row['Date Arrive'], row['Arrive Time Scheduled']), axis=1)
data['Actual Arrival DateTime'] = data.apply(lambda row: safe_to_datetime(row['Date Arrive'], row['Time Arrived']), axis=1)

# Calculate time differences
data['Time Difference'] = data['Actual Arrival DateTime'] - data['Scheduled DateTime']
data['Arrival Time Difference'] = (data['Time Difference'].dt.total_seconds() / 60)
data['Arrival Time Difference'] = data['Arrival Time Difference'].fillna(0).round().astype(int)

# Mask for large positive or negative values
mask_positive = data['Arrival Time Difference'] > 900
mask_negative = data['Arrival Time Difference'] < -900

# Adjust positive differences (flight departed next day)
data.loc[mask_positive, 'Arrival Time Difference'] = data['Arrival Time Difference'] - 1440

# Adjust negative differences (flight was scheduled on previous day but delayed to next)
data.loc[mask_negative, 'Arrival Time Difference'] = data['Arrival Time Difference'] + 1440

# Clean up unnecessary columns
data.drop(columns=['Time Difference', 'Actual Arrival DateTime', 'Scheduled DateTime'], inplace=True)

#### Estimation Time Difference

In [44]:
def hms_to_seconds(hms):
    try:
        h, m, s = map(int, hms.split(':'))
        return h * 3600 + m * 60 + s
    except Exception as e:
        return 0

# Convert 'Flight Duration' and 'Estimated Flight Duration' to total seconds
data['Flight Duration Seconds'] = data['Flight Duration'].apply(hms_to_seconds)
data['Estimated Flight Duration Seconds'] = data['Estimated Flight Duration'].apply(hms_to_seconds)

# Calculate the difference in seconds
data['Estimation Difference Seconds'] = data['Estimated Flight Duration Seconds'] - data['Flight Duration Seconds']

# Convert the difference to minutes
data['Estimation Difference Minutes'] = data['Estimation Difference Seconds'] / 60

# Round to the nearest whole number if needed
data['Estimation Difference Minutes'] = data['Estimation Difference Minutes'].round().astype(int)

# Drop temporary columns if needed
data.drop(columns=['Flight Duration Seconds', 'Estimated Flight Duration Seconds', 'Estimation Difference Seconds'], inplace=True)

#### Arrival Status

In [45]:
def classify_arrival_time_difference(diff):
    if diff == 0:
        return 'On Time'
    elif diff > 0:
        return 'Delayed'
    else:
        return 'Early'

# Apply the function to create the new column
data['Arrival Status'] = data['Arrival Time Difference'].apply(classify_arrival_time_difference)
data.loc[data['Status'] == 'CANCELLED', 'Arrival Status'] = 'Cancelled'

#### Departure Status

In [46]:
def classify_departure_time_difference(diff):
    if diff == 0:
        return 'On Time'
    elif diff > 0:
        return 'Delayed'
    else:
        return 'Early'

# Apply the function to create the new column
data['Departure Status'] = data['Departure Time Difference'].apply(classify_departure_time_difference)
data.loc[data['Status'] == 'CANCELLED', 'Departure Status'] = 'Cancelled'

## Export to SQL

In [47]:
# Step 1: Connect to the MySQL Database
connection = mysql.connector.connect(
    host="127.0.0.1",         # Database host (use "localhost" if running locally)
    user="root",              # Your MySQL username
    password="Lolablades1.",   # Your MySQL password
    database="Flights"         # Database name
)

cursor = connection.cursor()

# Step 2: Double-check the connection
if connection.is_connected():
    print("Connection to the database is successful!")
    
    # Step 3: Run a simple query to verify the connection
    try:
        cursor.execute("SHOW TABLES;")
        tables = cursor.fetchall()
        print("Tables in the database:", tables)
    except mysql.connector.Error as err:
        print(f"Error: {err}")
else:
    print("Connection failed.")

Connection to the database is successful!
Tables in the database: [('airlines_list',), ('arrivals',), ('berlin_arrivals',), ('berlin_departures',), ('departures',), ('heathrow_arrivals',), ('heathrow_departures',), ('lisbon_arrivals',), ('lisbon_departures',)]


In [48]:
data = data[~((data['Flight Duration'] == '00:00:00') & 
              (data['Origin'].isna()) & 
              (data['Country'].isna()))]
data = data[data['Airline'] != 'Find my terminal']
data['Date Depart'] = pd.to_datetime(data['Date Depart'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
data['Date Arrive'] = pd.to_datetime(data['Date Arrive'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
data['Time Departed'] = data['Time Departed'].replace(np.nan, None)
data['Time Arrived'] = data['Time Arrived'].replace(np.nan, None)

(656, 25)

(655, 25)

In [49]:
# Step 3: Create the table (if it doesn't exist) with additional columns adjusted for SQL types
create_table_query = """
CREATE TABLE IF NOT EXISTS heathrow_arrivals (
    id INT AUTO_INCREMENT PRIMARY KEY,
    flight_number VARCHAR(10),                  -- Flight number as VARCHAR
    airline VARCHAR(100),                       -- Airline as VARCHAR
    status VARCHAR(20),                         -- Flight status as VARCHAR
    origin VARCHAR(4),                          -- Origin airport IATA code as VARCHAR
    intersect VARCHAR(4),                       -- Intersect airport IATA code as VARCHAR
    destination VARCHAR(4),                     -- Destination airport IATA code as VARCHAR
    time_departed TIME,                         -- Time of departure as TIME
    departure_time_scheduled TIME,              -- Scheduled departure time as TIME
    terminal VARCHAR(8),                        -- Terminal as VARCHAR
    time_arrived TIME,                          -- Time of arrival as TIME
    arrive_time_scheduled TIME,                 -- Scheduled arrival time as TIME
    arrival_terminal VARCHAR(8),                -- Arrival terminal as VARCHAR
    day_code INT,                               -- Day code as INT
    date_arrive DATE,                           -- Arrival date as DATE
    date_depart DATE,                           -- Departure date as DATE
    departure_city VARCHAR(100),                -- Departure city as VARCHAR
    destination_city VARCHAR(100),              -- Destination city as VARCHAR
    country VARCHAR(100),                       -- Country as VARCHAR
    flight_duration TIME,                       -- Flight duration as TIME
    estimated_flight_duration TIME,             -- Estimated flight duration as TIME
    departure_time_difference INT,              -- Difference in departure time as INT
    arrival_time_difference INT,                -- Difference in arrival time as INT
    estimation_difference_minutes INT,          -- Estimation difference in minutes as INT
    arrival_status VARCHAR(20),                  -- Arrival status as VARCHAR
    departure_status VARCHAR(20)               -- Departure status as VARCHAR
);
"""
cursor.execute(create_table_query)

# Step 4: Prepare SQL query to insert data
insert_query = """
INSERT INTO heathrow_arrivals (flight_number, airline, status, origin, intersect, destination, time_departed,
                     departure_time_scheduled, terminal, time_arrived, 
                     arrive_time_scheduled, arrival_terminal, day_code, date_arrive, date_depart, 
                     departure_city, destination_city, country, flight_duration, 
                     estimated_flight_duration, departure_time_difference, arrival_time_difference, 
                     estimation_difference_minutes, arrival_status, departure_status)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Step 5: Iterate over the DataFrame and insert each row into the table
for i, row in data.iterrows():
    try:
        cursor.execute(insert_query, (
            row['Flight Number'], 
            row['Airline'],
            row['Status'], 
            row['Origin'], 
            row['Intersect'], 
            row['Destination'], 
            row['Time Departed'], 
            row['Departure Time Scheduled'], 
            row['Terminal'], 
            row['Time Arrived'], 
            row['Arrive Time Scheduled'], 
            row['Arrival Terminal'], 
            row['Day Code'], 
            row['Date Arrive'], 
            row['Date Depart'], 
            row['Departure City'], 
            row['Destination City'], 
            row['Country'], 
            row['Flight Duration'], 
            row['Estimated Flight Duration'], 
            row['Departure Time Difference'], 
            row['Arrival Time Difference'], 
            row['Estimation Difference Minutes'], 
            row['Arrival Status'], 
            row['Departure Status']
        ))
    except mysql.connector.Error as err:
        print(f"Error at row {i}: {err}")

# Step 6: Commit the transaction and close the connection
connection.commit()

cursor.close()
connection.close()

print("CSV file uploaded successfully!")

CSV file uploaded successfully!


---